# プラグインエージェントを使用したマルチエージェントシステム

## 概要

このノートブックでは、Semantic Kernel のプラグイン機能を活用してマルチエージェントシステムを構築する方法を学習します。

### 主な学習内容

1. **カスタムプラグインの作成**
   - kernel_function デコレータの使用
   - 時間・天気・温度変換・メール送信の各プラグイン

2. **専門エージェントの作成**
   - 特定の機能に特化したエージェント
   - プラグインとエージェントの統合

3. **トリアージエージェントによる統制**
   - ユーザーリクエストの解析と適切なエージェントへの転送
   - 複数エージェントの連携制御

# ライブラリのインポート

プラグインエージェントシステムの構築に必要なライブラリをインポートします。

## 主要コンポーネント

- **kernel_function**: カスタムプラグイン関数の定義
- **ChatCompletionAgent**: 各専門エージェントの基盤
- **Annotated**: 関数パラメータの型注釈とドキュメント

In [1]:
import os
import json
import asyncio
import datetime

from dotenv import load_dotenv, find_dotenv

from IPython.display import Image, display

from azure.identity.aio import DefaultAzureCredential
from azure.ai.agents.models import (
    FileInfo, FileSearchTool, VectorStore,
    CodeInterpreterTool, FilePurpose,
    ListSortOrder
)

from semantic_kernel.agents import (
    ChatCompletionAgent, ChatHistoryAgentThread,
    AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    ChatMessageContent, FunctionCallContent, FunctionResultContent, AuthorRole, TextContent
)

import json
from typing import Annotated
import datetime
from semantic_kernel.functions import kernel_function

# 環境変数の取得

Azure OpenAI Service への接続に必要な認証情報を環境変数から取得します。

In [2]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")

# ユーティリティ関数

エージェントとの対話をデバッグ・監視するためのヘルパー関数を定義します。

In [3]:
async def print_thread_message_details(thread: str):
    """
    スレッドのメッセージ詳細を表示します。

    Args:
        thread (str): スレッドのインスタンス
    """
    async for message in thread.get_messages():
        print("-----")

        for item in message.items:
            if isinstance(item, FunctionCallContent):
                print(f"[Function Calling] by {message.ai_model_id}")
                print(f" - Function Name : {item.name}")
                print(f" - Arguments     : {item.arguments}")

            elif isinstance(item, FunctionResultContent):
                print(f"[Function Result]")
                # 文字列のデコード変換
                if isinstance(item.result, str):
                    try:
                        decoded = json.loads(item.result)
                        print(f" - Result        : {decoded}") # デコード成功時は変換後の値を表示
                    except json.JSONDecodeError:
                        print(f" - Result        : {item.result}")  # デコード失敗時はそのまま
                else:
                    print(f" - Result        : {item.result}")

            elif isinstance(item, TextContent):
                if message.name:
                    print(f"[Agent Response] from {message.ai_model_id}")
                else:
                    print("[User Message]")
                print(f" - Content       : {item.text}")

            else:
                print(f"[Unknown Item Type] ({type(item).__name__})")
                print(f" - Raw Item      : {item}")


def log_with_timestamp(message: str) -> None:
    """
    現在時刻付きでメッセージを標準出力にログとして表示します。

    Args:
        message (str): 出力するログメッセージ。
    """
    timestamp = datetime.datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"[{timestamp}] {message}")

# プラグインの作成

各専門機能を提供するカスタムプラグインクラスを定義します。

## 提供プラグイン

- **TimeWeatherPlugin**: 現在時刻と天気情報の取得
- **ConvertTemperaturePlugin**: 摂氏から華氏への温度変換
- **SendEmailPlugin**: メール送信機能

In [4]:
class TimeWeatherPlugin:

    @kernel_function(
        name="fetch_current_datetime",
        description="現在の時刻を JSON 文字列として取得します。オプションでフォーマットを指定できます。",
    )
    def fetch_current_datetime(
        self,
        format: Annotated[str, "現在の時刻を返す形式（例: '%Y/%m/%d %H:%M'）。未指定時はデフォルト形式。"] = "",
    ) -> Annotated[str, "現在の時刻を含む JSON 文字列（例: {'current_time': '2023-10-01 12:00:00'}）"]:

        time_format = format or "%Y-%m-%d %H:%M:%S"
        current_time = datetime.datetime.now().strftime(time_format)
        return json.dumps({"current_time": current_time})


    @kernel_function(
        name="fetch_weather",
        description="指定された場所の天気情報を取得します。",
    )
    def fetch_weather(
        self, 
        location: Annotated[str, "天気情報を取得する都市名（例: Tokyo, New York, London）"]
    ) -> Annotated[str, "天気情報を含む JSON 文字列（例: {'weather': 'Sunny, 25°C'}）"]:

        # ダミーの天気データを使用
        dummy_weather_data = {
            "New York": "Sunny, 25°C",
            "London": "Cloudy, 18°C",
            "Tokyo": "Rainy, 22°C"
        }
        weather = dummy_weather_data.get(
            location, "Weather data not available for this location."
        )

        return json.dumps({"weather": weather})


class ConvertTemperaturePlugin:

    @kernel_function(
        name="convert_temperature",
        description="温度を摂氏から華氏に変換します。",
    )
    def convert_temperature(
        self,
        celsius: Annotated[float, "摂氏温度（例: 25.0）"]
    ) -> Annotated[str, "華氏温度を含む JSON 文字列（例: {'fahrenheit': 77.0}）"]:

        fahrenheit = (celsius * 9 / 5) + 32

        return json.dumps({"fahrenheit": fahrenheit})


class SendEmailPlugin:

    @kernel_function(
        name="send_email",
        description="指定の件名と本文を含むメールを宛先に送信します。",
    )
    def send_email(
        self,
        recipient: Annotated[str, "メールの宛先アドレス（例: user@example.com）"],
        subject: Annotated[str, "メールの件名"],
        body: Annotated[str, "メールの本文"]
    ) -> Annotated[str, "完了通知を含む文字列（例: {'message': Email successfully sent to xxx@example.com.}）"]:
        print(f"Sending email to {recipient}...")
        print(f"Subject: {subject}")
        print(f"Body:\n{body}")
        return json.dumps({"message": f"Email successfully sent to {recipient}."})

# クライアントの初期化

Azure OpenAI Service への接続クライアントを初期化します。

In [5]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# マルチエージェントの作成

特定の機能に特化した専門エージェントを作成します。

# プラグインエージェントの作成

各プラグインを組み込んだ専門エージェントを作成し、それぞれに特化した役割を割り当てます。

In [6]:
# 専門エージェントの作成
time_weather_agent = ChatCompletionAgent(
    service=azure_completion_service, 
    name="TimeWeatherAgent", 
    instructions="あなたは時間と天気のクエリの専門的なエージェントです。",
    plugins=[TimeWeatherPlugin()]
)

temperature_agent = ChatCompletionAgent(
    service=azure_completion_service, 
    name="TemperatureAgent", 
    instructions="あなたは温度変換の専門的なエージェントです。",
    plugins=[ConvertTemperaturePlugin()]
)

send_email_agent = ChatCompletionAgent(
    service=azure_completion_service, 
    name="SendEmailAgent", 
    instructions="あなたは電子メールを送信するための専門のエージェントです。",
    plugins=[SendEmailPlugin()]
)

# 親エージェントの作成

ユーザーのリクエストを解析し、適切な専門エージェントに転送するトリアージエージェントを作成します。

In [7]:
triage_agent = ChatCompletionAgent(
    service=azure_completion_service, 
    name="TriageAgent",
    instructions=(
        "ユーザーのリクエストを評価し、適切なエージェント（TimeWeatherAgent, TemperatureAgent, SendEmailAgent ）"
        "に転送して、適切なサポートを提供します。エージェントからの情報を含め、ユーザーに完全な回答を提供します。"
        "元のユーザーリクエストが完全に処理されたどうか確認してください。"
    ),
    plugins=[time_weather_agent, temperature_agent, send_email_agent],
)

In [8]:
formatted_json = json.dumps(triage_agent.model_dump(), indent=2, ensure_ascii=False, default=str)
print(formatted_json)

{
  "arguments": null,
  "description": null,
  "id": "5d99108e-a4b6-490a-bf17-320bd89651c1",
  "instructions": "ユーザーのリクエストを評価し、適切なエージェント（TimeWeatherAgent, TemperatureAgent, SendEmailAgent ）に転送して、適切なサポートを提供します。エージェントからの情報を含め、ユーザーに完全な回答を提供します。元のユーザーリクエストが完全に処理されたどうか確認してください。",
  "kernel": {
    "services": {
      "azure_completion_agent": {
        "ai_model_id": "gpt-4.1",
        "service_id": "azure_completion_agent"
      }
    },
    "ai_service_selector": "<semantic_kernel.services.ai_service_selector.AIServiceSelector object at 0x000001C702E0DE50>",
    "plugins": {
      "TimeWeatherAgent": {
        "name": "TimeWeatherAgent",
        "description": null,
        "functions": {
          "TimeWeatherAgent": {
            "metadata": {
              "name": "TimeWeatherAgent",
              "plugin_name": "TimeWeatherAgent",
              "description": "あなたは時間と天気のクエリの専門的なエージェントです。",
              "parameters": [
                {
                  "name": "messages",
         

# 動作確認

構築したマルチエージェントシステムの動作を確認します。複数ステップの処理フローを実行します。

In [9]:
# スレッドの作成
thread = ChatHistoryAgentThread()

In [10]:
user_input = (
    "はじめに、現在の時刻を '%Y-%m-%d %H:%M:%S' 形式で、また Tokyo の天気を教えてください。"
    "次に、Tokyo の気温を華氏に変換してください。"
    "最後に、結果の概要を記載したメールをサンプル受信者に送信してください。"
)

# エージェントの実行＆レスポンスの取得
response = await triage_agent.get_response(
    messages=user_input,
    thread=thread
)



Sending email to サンプル受信者様...
Subject: 現在時刻と天気のご案内
Body:
サンプル受信者様

現在の時刻は2025-07-30 04:06:23です。
Tokyoの天気は雨で、気温は22°C（71.6°F）です。
ご参考までに概要をお送りいたします。


In [11]:
await print_thread_message_details(thread)

-----
[User Message]
 - Content       : はじめに、現在の時刻を '%Y-%m-%d %H:%M:%S' 形式で、また Tokyo の天気を教えてください。次に、Tokyo の気温を華氏に変換してください。最後に、結果の概要を記載したメールをサンプル受信者に送信してください。
-----
[Function Calling] by gpt-4.1
 - Function Name : TimeWeatherAgent-TimeWeatherAgent
 - Arguments     : {"messages":"現在の時刻を '%Y-%m-%d %H:%M:%S' 形式で、また Tokyo の天気と気温を教えてください。"}
-----
[Function Result]
 - Result        : 現在の時刻は 2025-07-30 04:06:23 です。
Tokyoの天気は雨、気温は22°Cです。
-----
[Function Calling] by gpt-4.1
 - Function Name : TemperatureAgent-TemperatureAgent
 - Arguments     : {"messages":"22°C を華氏に変換してください。"}
-----
[Function Result]
 - Result        : 22°Cは華氏で71.6°Fです。
-----
[Function Calling] by gpt-4.1
 - Function Name : SendEmailAgent-SendEmailAgent
 - Arguments     : {"messages":"サンプル受信者様\n\n現在の時刻は2025-07-30 04:06:23です。Tokyoの天気は雨で、気温は22°C（71.6°F）です。\nご参考までに概要をお送りいたします。"}
-----
[Function Result]
 - Result        : 以下の内容でメールを送信しました。

---
件名: 現在時刻と天気のご案内

本文:
サンプル受信者様

現在の時刻は2025-07-30 04:06:23です。
Tokyoの天気は雨で、気温は22°C（71.6°F）で